In [140]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.datasets import load_wine

In [141]:
data = load_wine()
X_data = data.data
y_data = data.target
n_sample, n_feature = X_data.shape

In [142]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_label = lb.fit(y_data)
y_data0 = lb.fit_transform(y_data)

In [143]:
X_tensor = torch.Tensor(X_data.astype(np.float32))
y_tensor = torch.Tensor(y_data0)
dataset = TensorDataset(X_tensor, y_tensor)

test_ratio = 0.2  # 20% data split into test dataset
test_size = int(n_sample * test_ratio)
train_size = n_sample - test_size
train_ds, test_ds = random_split(dataset,[train_size, test_size])
train_dataloader = DataLoader(train_ds, batch_size=train_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=test_size)

In [181]:
def accuracy(true_val, pred_val):
    t0 = ((true_val.argmax(dim=1) == pred_val.argmax(dim=1))*1)
    return t0.numpy().mean()

In [197]:
import os
class NNmodel(nn.Module):
    def __init__(self,input_size,hid_size,output_size):
        super(NNmodel,self).__init__()
        self.out = nn.Linear(input_size,hid_size,bias=True)
        self.out2 = nn.Linear(hid_size,output_size,bias=True)
    def forward(self,x):
        z = torch.sigmoid(self.out(x))
        z = torch.sigmoid(self.out2(z))
        return z

for x,y in train_dataloader:
    train_x = x
    train_y = y

for x,y in test_dataloader:
    test_x = x
    test_y = y
file_name = 'NN_13_8_3.pt'
model = NNmodel(n_feature,8,3)
optimizer = torch.optim.Adam(model.parameters(),lr =1e-4)
loss = nn.BCELoss()
iter = int(5e4)

if os.path.exists(file_name):
    model.load_state_dict(torch.load(file_name))
    iter = 1000


for i in range(iter):
    z = model(train_x)
    J = loss(z,train_y)
    J.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i%(iter/10)==0:
        with torch.no_grad():
            zx = model(test_x)
            Jx = loss(zx,test_y)
            acc_test = accuracy(zx,test_y)*100
        acc_train = accuracy(z,train_y)*100
        
        print(f'Epoch {i}: Train cost = {J.item():.5f}, Test cost = {Jx.item():.5f}, Train acc = {acc_train:.3f}%, Test acc = {acc_test:.3f}%')
torch.save(model.state_dict(),file_name)

Epoch 0: Train cost = 0.04076, Test cost = 0.12084, Train acc = 98.601%, Test acc = 97.143%
Epoch 100: Train cost = 0.04076, Test cost = 0.11593, Train acc = 98.601%, Test acc = 97.143%
Epoch 200: Train cost = 0.04076, Test cost = 0.11571, Train acc = 98.601%, Test acc = 97.143%
Epoch 300: Train cost = 0.04076, Test cost = 0.11560, Train acc = 98.601%, Test acc = 97.143%
Epoch 400: Train cost = 0.04076, Test cost = 0.11547, Train acc = 98.601%, Test acc = 97.143%
Epoch 500: Train cost = 0.04076, Test cost = 0.11518, Train acc = 98.601%, Test acc = 97.143%
Epoch 600: Train cost = 0.04076, Test cost = 0.11500, Train acc = 98.601%, Test acc = 97.143%
Epoch 700: Train cost = 0.04076, Test cost = 0.11482, Train acc = 98.601%, Test acc = 97.143%
Epoch 800: Train cost = 0.04076, Test cost = 0.11467, Train acc = 98.601%, Test acc = 97.143%
Epoch 900: Train cost = 0.04076, Test cost = 0.11443, Train acc = 98.601%, Test acc = 97.143%
